In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [2]:
eve = input()
url = "https://www.vlr.gg/event/" + eve
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [3]:
t_p = []
for link in soup.find_all('a', class_={'wf-module-item'}):
    t_p.append(link.get('href'))

teams = []
players = []

current_team = None
for item in t_p:
    if '/team/' in item:
        current_team = re.search(r'/team/\d+/(.+)', item)
        if current_team:
            current_team = current_team.group(1)
    else:
        player_match = re.search(r'/player/\d+/(.+)', item)
        if player_match:
            players.append(player_match.group(1))
            teams.append(current_team)

# Créer un DataFrame
team_player = pd.DataFrame({'Team': teams, 'Player': players})

# Afficher le DataFrame
team_player

,Team,Player
0,papara-supermassive,sterben
1,papara-supermassive,izzy
2,papara-supermassive,cyderx
3,papara-supermassive,hiro
4,papara-supermassive,celasun
5,galakticos,jlerst
6,galakticos,destruct1vee
7,galakticos,glovee
8,galakticos,ip0tt
9,galakticos,krostaly


In [12]:
matchs = []
for link in soup.find_all('a', class_={'bracket-item'}):
    matchs.append(link.get('href'))
    
team_names = soup.find_all('div', class_={'bracket-item-team-name'})
name = [team_name.get_text(strip=True) for team_name in team_names]
team_scores = soup.find_all('div', class_={'bracket-item-team-score'})
scores = [team_score.get_text(strip=True) for team_score in team_scores]

num_mat = []
mat = []
for item in matchs:
    match = re.match(r'/(?P<number>\d+)/(?P<text>.+)', item)
    if match:
        num_mat.append(match.group(1))

for link in soup.find_all('a', class_={'bracket-item'}):
    mat.append(link.get('title'))

# Créer un DataFrame
df = pd.DataFrame({'Numéro de match': num_mat, 'Match': mat, 'Team 1': name[::2], 'Score Team 1': scores[::2], 'Team 2':name[1::2], 'Score Team 2': scores[1::2]})

# Afficher le DataFrame
df

,Numéro de match,Match,Team 1,Score Team 1,Team 2,Score Team 2
0,292514,Digital Athletics vs. Regnum Carya Esports,Digital Athletics,2,Regnum Carya Esports,0
1,292515,Fenerbahçe Espor vs. Fire Flux Esports,Fenerbahçe Espor,0,Fire Flux Esports,2
2,292516,Galakticos vs. Galatasaray Espor,Galakticos,2,Galatasaray Espor,0
3,292517,İstanbul Wildcats vs. Papara SuperMassive,İstanbul Wildcats,2,Papara SuperMassive,1
4,292518,Digital Athletics vs. Fire Flux Esports,Digital Athletics,0,Fire Flux Esports,2
5,292519,Galakticos vs. İstanbul Wildcats,Galakticos,1,İstanbul Wildcats,2
6,292520,Fire Flux Esports vs. İstanbul Wildcats,Fire Flux Esports,2,İstanbul Wildcats,1
7,292521,Fire Flux Esports vs. Fenerbahçe Espor,Fire Flux Esports,3,Fenerbahçe Espor,0
8,292522,Regnum Carya Esports vs. Fenerbahçe Espor,Regnum Carya Esports,0,Fenerbahçe Espor,2
9,292523,Galatasaray Espor vs. Papara SuperMassive,Galatasaray Espor,2,Papara SuperMassive,1


In [ ]:
soup